In [ ]:
import sys
sys.path.append('../../')

import estraces
import scared
from kresca.plot import CurvePlot
from kresca.reverse import LeakageAssessment

## Import Traces

In [ ]:
ths = estraces.read_ths_from_ets_file('../../Traces/DES_1k.ets')

In [ ]:
print(ths)

## Plot Traces

In [ ]:
#put inline=True if you want to display on notebook
CurvePlot(ths.samples[:5], inline=True) 

## Make Selection Funtions

In [ ]:
@scared.reverse_selection_function
def plaintext(plaintext):
    return plaintext

@scared.reverse_selection_function
def first_addroundkey(plaintext, key):
    return scared.des.encrypt(plaintext, key, at_round=0, after_step=scared.des.Steps.ADD_ROUND_KEY)

@scared.reverse_selection_function
def first_sboxes(plaintext, key):
    return scared.des.encrypt(plaintext, key, at_round=0, after_step=scared.des.Steps.SBOXES)

@scared.reverse_selection_function
def first_feistel(plaintext, key):
    return scared.des.encrypt(plaintext, key, at_round=0, after_step=scared.des.Steps.INV_PERMUTATION_P_RIGHT)

@scared.reverse_selection_function
def last_addroundkey(ciphertext, key):
    return scared.des.encrypt(ciphertext, key, at_round=0, after_step=scared.des.Steps.ADD_ROUND_KEY)

@scared.reverse_selection_function
def last_sboxes(ciphertext, key):
    return scared.des.encrypt(ciphertext, key, at_round=0, after_step=scared.des.Steps.SBOXES)

@scared.reverse_selection_function
def last_feistel(ciphertext, key):
    return scared.des.encrypt(ciphertext, key, at_round=0, after_step=scared.des.Steps.INV_PERMUTATION_P_RIGHT)

@scared.reverse_selection_function
def ciphertext(ciphertext):
    return ciphertext

In [ ]:
DES_selection_functions = {
    'DES Plaintext': plaintext,
    'DES First Add Round Key': first_addroundkey,
    'DES First Sboxes': first_sboxes,
    'DES First Feistel': first_feistel,
#     'DES Last Add Round Key': last_addroundkey,
#     'DES Last Sboxes': last_sboxes,
#     'DES Last Feistel': last_feistel,
#     'DES Ciphertext': ciphertext,
}

## Make Leakage Assessment Object

In [ ]:
reverse = LeakageAssessment(
    ths=ths,
    selection_functions=DES_selection_functions,
#     distinguisher=scared.CPAReverse,
#     model=scared.HammingWeight(),
#     frame=range(20000, 30000),
#     preprocesses=[]
)

In [ ]:
reverse.run()

In [ ]:
reverse.show_result()